In [24]:
from selenium import webdriver
import time
import pandas as pd
import os
import re

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

import spacy

In [8]:
service = Service(executable_path='./chromedriver-mac-arm64/chromedriver')
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)

In [ ]:
def linkedin_login():
    linkedinHomePage = "https://www.linkedin.com/login"
    driver.get(linkedinHomePage)
    driver.implicitly_wait(3)
    
    username = driver.find_element(By.ID,"username")
    password = driver.find_element(By.ID,"password")
    
    your_username = #enter username
    your_password = #enter password
    
    username.send_keys(your_username)
    password.send_keys(your_password)
    
    login_button = driver.find_element(By.XPATH,"//button[@type='submit']")
    login_button.click()
    print("Logged In")

linkedin_login()

Logged In


In [10]:
def job_listings(url):
    driver.get(url)
    num_of_results = driver.find_elements(By.CLASS_NAME,"jobs-search-results-list__subtitle")[0].text  
    num_of_results = num_of_results.split()[0]
    num_of_results = int(num_of_results.replace(',',''))

    number_of_posting_per_page = 25
    number_of_pages = num_of_results//number_of_posting_per_page + 1

    print("Number of Jobs : ",num_of_results)
    print("Number of Pages : ",number_of_pages)

url = "https://www.linkedin.com/jobs/search/?currentJobId=4129461199&geoId=102713980&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"
job_listings(url)


Number of Jobs :  7340
Number of Pages :  294


In [23]:
#Load NLP Model - Skill extraction from JD
ner_model = spacy.load(r"data preprocessing/output/model-best")

In [12]:
def extractskills(job_description):
    exp = ""
    doc = ner_model(job_description)
    all_skills = set()
    for ent in doc.ents:
        if(ent.label_ == "EXPERIENCE"):
            skill = ent.text.strip().capitalize()
            exp = ent.text
        elif(ent.label_ == "SKILLS" or ent.label_ == "SOFT-SKILLS"):
            skill = ent.text.strip().capitalize()
            all_skills.add(skill)
    
    return exp,all_skills

In [13]:
def extractInfo(table1,table2):
    company_name = driver.find_elements(By.CLASS_NAME,"job-details-jobs-unified-top-card__company-name")[0].text
    profile = driver.find_elements(By.CLASS_NAME,"job-details-jobs-unified-top-card__job-title")[0].text
    profile = profile.strip().capitalize()
    
    job_description = driver.find_elements(By.CLASS_NAME,"jobs-box__html-content")[0].text
    experience,skills = extractskills(job_description)
    if(not skills):
        return table1,table2

    record1 = pd.DataFrame({'Company': company_name,
                            'Profile': profile,
                            'Experience':experience},
                            index=[len(table1)])
    table1 = pd.concat([table1, record1], ignore_index=True)

    for skill in skills:
        record2 = pd.DataFrame({'f_key':len(table1)-1,'Skills':skill},index=[len(table2)])
        table2 = pd.concat([table2, record2], ignore_index=True)

    return table1,table2

In [ ]:
def scrap_and_create_table(num_pages = 9):
    table1 = pd.DataFrame(columns=["Company","Profile","Experience"])
    table2 = pd.DataFrame(columns=["f_key","Skills"])

    if(num_pages > 9):
        num_pages1 = 9
        num_pages2 = num_pages - 9
    else:
        num_pages1 = num_pages
        num_pages2 = 0

    for i in range(num_pages1):
        #page-flip
        driver.find_elements(By.CLASS_NAME,"artdeco-pagination__indicator--number")[i].click()
        time.sleep(3)

        #Select All Job Description
        all_job_list = driver.find_elements(By.CLASS_NAME,"job-card-list__entity-lockup")

        #Extract Info
        for j in range(len(all_job_list)):
            all_job_list[j].click()
            time.sleep(1)

            table1,table2 = extractInfo(table1,table2)

    for _ in range(9,num_pages2):
        driver.find_elements(By.CLASS_NAME,"artdeco-pagination__indicator--number")[-4].click()
        time.sleep(3)

        #Select All Job Description
        all_job_list = driver.find_elements(By.CLASS_NAME,"job-card-list__entity-lockup")

        #Extract Info
        for j in range(len(all_job_list)):
            all_job_list[j].click()
            time.sleep(1)

            table1,table2 = extractInfo(table1,table2)

    table1 = table1["Experience"].apply(lambda text: int(min([int(i) for i in re.findall(r'\d+', text)])) if text else 0)
    
    return table1,table2


In [15]:
table1,table2 = scrap_and_create_table(6)

In [ ]:
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from fuzzywuzzy import fuzz
from scipy import stats


def MapToProfile(table1,job_profiles):
    n_clusters = len(job_profiles)

    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform([profile for profile in table1["Profile"]])

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X_train)

    labels = kmeans.labels_

    #Random selection of few  same label and map to the following list
    dic = {}
    for label_id in range(n_clusters):
        record_labels = np.where(labels == label_id)[0]
        sampling_size = 10
        record_labels = record_labels[np.random.choice(len(record_labels),sampling_size)]

        voting = []
        for string1 in table1["Profile"].iloc[record_labels]:
            voting1 = [fuzz.ratio(string1, string2) for string2 in job_profiles]
            voting.append(np.argmax(voting1))
        
        mappedKeyword_idx = int(stats.mode(voting)[0])
        dic[label_id] = mappedKeyword_idx
    
    #Add new Column to table1
    filtered_names = []
    for label in labels:
        filtered_names.append(job_profiles[dic[label]])
    
    table1["Filtered_Profile"] = filtered_names
    return table1

In [162]:
job_profiles = ["Data scientist","Data analyst","Business analyst"]

table1 = MapToProfile(table1,job_profiles)

{0: 1, 1: 0, 2: 2}


In [ ]:
with pd.ExcelWriter('Company.xlsx') as writer:  
    table1.to_excel(writer, sheet_name='Company')
    table2.to_excel(writer, sheet_name='Skills')

In [33]:
import sqlite3
import pandas as pd

cnx = sqlite3.connect('Company.db')

#df1 = pd.read_sql_query("SELECT * FROM Company", cnx)
#df2 = pd.read_sql_query("SELECT * FROM Skills", cnx)

with pd.ExcelWriter('Company.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Company')
    df2.to_excel(writer, sheet_name='Skills')

In [31]:
df1["Experience"] = df1["Experience"].apply(lambda text: int(min([int(i) for i in re.findall(r'\d+', text)])) if text else 0)
    

In [32]:
df1.head()

,index,Company,Profile,Experience,Filtered_Profile
0,0,Third Bridge Creative,Music data evaluator (india),2,Data analyst
1,1,OptimSpace,Data scientist intern,0,Data scientist
2,2,TwiLearn,Data scientist intern,0,Data scientist
3,3,Microsoft,Data science internship opportunities: second-...,0,Data analyst
4,4,Peroptyx,Data analyst,0,Data analyst
